## Determine main melody 
 - by score (author specified)
 - pan (左右聲道平衡度)
 - velociy (按鍵力道)
 - 平均旋律
 - 和弦所佔比率

In [1]:
main_name = ['melody','melodie','vocal','voice','sing','lead']
sub_name = ['drum']

## pan value

In [2]:
def panDect(parts):
    pan = {}
    for part in parts:
        staffs = part.select('Staff')
        for staff in staffs:
            i = staff.get('id')
            channel = part.select('Channel')[0].select('controller')
            value = 0
            if len(channel) == 0 or len(channel.select('controller')) == 0 or channel.select('controller')[0].get('ctrl') != '10':
                value = 63
            else:
                value = channel.select('controller')[0].get('value')
            pan.update({i:value})
    return pan

## average velocity

In [3]:
def velDect(addr,ids):
    fm = open(addr, 'r')
    music = fm.read()
    fm.close()
    vela = {}
    staffs = [staff for staff in bs(music,'xml').select('Score > Staff') if staff.get('id') in ids]
    for staff in staffs:
        vels = staff.select('Note velocity')
        n = len(vels)
        s = 0.0
        for vel in vels:
            v = int(vel.text)
            s += v
        avg = s/n
        vela.update({staff.get('id'):str(avg)})
    return vela

## average pitch

In [4]:
import math
def pitDect(addr,ids):
    fm = open(addr, 'r')
    music = fm.read()
    fm.close()
    pits = {}
    dutype = {'whole':1.0,'half':1.0/2,'quarter':1.0/4,'eighth':1.0/8,'16th':1.0/16,'32nd':1.0/32,'64th':1.0/64,'128th':1.0/128}
    staffs = [staff for staff in bs(music,'xml').select('Score > Staff') if staff.get('id') in ids]
    for staff in staffs:
        pitch_sum = 0
        pitch_time = 0
        for measure in staff.select('Measure'):
            for chord in [c for c in list(measure.children) if c != '\n' and c.name == 'Chord']:
                max_pitch = max([int(d.pitch.text) for d in chord.select('Note')])
                dot = 0
                if len(chord.select('dot')) != 0:
                    dot = int(chord.select('dot')[0].text)
                dutime = dutype[chord.durationType.text] * (2-math.pow(1.0/2,dot))
                pitch_sum += max_pitch * dutime
                pitch_time += dutime
        avg_pich = pitch_sum/pitch_time
        pits.update({staff.get('id'):str(avg_pich)})
    return pits

## chords' rate

In [5]:
def rateDect(addr,ids):
    fm = open(addr, 'r')
    music = fm.read()
    fm.close()
    rates = {}
    staffs = [staff for staff in bs(music,'xml').select('Score > Staff') if staff.get('id') in ids]
    for staff in staffs:
        num = 0
        n = 0
        for measure in staff.select('Measure'):
            chords = [c for c in list(measure.children) if c != '\n' and c.name == 'Chord']
            n += len(chords)
            for chord in chords:
                if len(chord.select('Note')) == 1:
                    num += 1
        rates.update({staff.get('id'):str(float(num)/n)})
    return rates

## sound area

In [6]:
def areaDect(addr,ids):
    fm = open(addr, 'r')
    music = fm.read()
    fm.close()
    area = {}
    staffs = [staff for staff in bs(music,'xml').select('Score > Staff') if staff.get('id') in ids]
    dutype = {'whole':1.0,'half':1.0/2,'quarter':1.0/4,'eighth':1.0/8,'16th':1.0/16,'32nd':1.0/32,'64th':1.0/64,'128th':1.0/128}
    for staff in staffs:
        du = 0
        all_du = []
        for measure in staff.select('Measure'):
            chords = [c for c in list(measure.children) if c != '\n' and c.name == 'Chord']
            for chord in chords:
                all_du.append(dutype[chord.durationType.text])
        area.update({staff.get('id'):str(sum(all_du))})
    return area

## determine main melodies

In [7]:
import pandas as pd
def detMain(data):
    form = pd.DataFrame(data).transpose()
    form.columns = ['pan','vel_rate','pit_rate','cho_rate','area','ismaster1']
    # need to add...
    return form       

### isMaster:判斷LONGNAME裡是否有main_name的字，有給1沒有給0

In [8]:
#isMaster
def isMaster(parts):#這裡的parts是main function裡的cand
    ismaster = {}
    for part in parts:
        staffs = part.select('Staff')#找每個樂器裡面的Staff
        for staff in staffs:
            i = staff.get('id') 
            value=0#value先給值:0
            pname = part.select('longName')[0].text.encode('utf8').lower()            
            if len([word for word in main_name if word in pname]) != 0: # instru. name
                value=1#如果pname裡面有main_name的字，value改給值:1
            ismaster.update({i:value})
    return ismaster

## main function

In [164]:
#版本1用isMaster找出LONGNAME裡面有在字典MAIN_NAME的給1沒有的給0
from bs4 import BeautifulSoup as bs
# addr = 'E:/musicteam/music/Believe.mscx'
addr = 'E:/musicteam/music/Again.mscx'
fm = open(addr, 'r')
music = fm.read()
fm.close()
cand = bs(music,'xml').select('Part')
main_cand = []
sub_cand = []
for s in [par for par in cand if len(par.select('longName')) != 0]:
    pname = s.select('longName')[0].text.encode('utf8').lower()
    if len([word for word in main_name if word in pname]) != 0: # instru. name
        main_cand.append(s)        
#         cand.remove(s)
    if len([word for word in sub_name if word in pname]) != 0:
        sub_cand.append(s)
        cand.remove(s)
    for cin in [c for c in list(s.select('Instrument')[0].children) if c != '\n']: # more than one instru. in this staff
        if cin.get('pitch') is not None:
            if s in cand:
                sub_cand.append(s)
                cand.remove(s)
            break
mesg = {}
ismaster = isMaster(cand)
pan = panDect(cand)
ids = pan.keys()
vels = velDect(addr,ids)
pits = pitDect(addr,ids)
rates = rateDect(addr,ids)
area = areaDect(addr,ids)
#print pan.keys(), area.keys(),ismaster1.keys(),ismaster1
#print len(pan),len(vels),len(pits),len(rates),len(area),len(ismaster1)
[mesg.update({a:[pan[a],vels[a],pits[a],rates[a],area[a],ismaster[a]]}) for a in pan.keys()]
f = detMain(mesg)
print f

[u'11', u'10', u'13', u'12', u'14', u'1', u'3', u'2', u'5', u'7', u'6', u'9', u'8'] [u'11', u'10', u'13', u'12', u'14', u'1', u'3', u'2', u'5', u'7', u'6', u'9', u'8'] [u'1', u'3', u'5', u'4', u'7', u'6', u'9', u'8'] {u'1': 1, u'3': 0, u'5': 0, u'4': 0, u'7': 0, u'6': 0, u'9': 1, u'8': 0}
13 13 13 13 13 8
   pan       vel_rate       pit_rate        cho_rate       area ismaster1
1   63  101.384615385  64.6890418818             0.7   54.46875         0
10  63  70.7823834197  56.4756918474   0.85119047619    83.5625         0
11  63  103.706231454  63.2879455346  0.994029850746  39.015625         1
12  63           65.6           63.8            0.75       6.25         0
13  63           74.4          42.75  0.636363636364        8.0         0
14  63   75.914893617  71.1922428331             1.0   18.53125         0
2   63  98.2965779468  50.6162162162  0.768115942029    57.8125         0
3   63   76.795379538  73.0679811049  0.616504854369  76.078125         0
5   63  89.0588235294   58.

### main function2 最後決定用這個跑測試資料的CSV


In [9]:
#版本2與版本1一樣，不同於沒用isMaster
from bs4 import BeautifulSoup as bs
import os

fi = open('E:/musicteam/musictxt/test2.csv','w')
for filename in os.listdir('E:/musicteam/music'):
    try:
        addr = 'E:/musicteam/music/'+filename
        fm = open(addr, 'r')
        music = fm.read()
        fm.close()
        cand = bs(music,'xml').select('Part')
        main_cand = []
        sub_cand = []
        ismaster = {}
        for s in [par for par in cand if len(par.select('longName')) != 0]:
            pname = s.select('longName')[0].text.encode('utf8').lower()
            for staff in s.select('Staff'):
                i = staff.get('id') 
                value=0
                if len([word for word in main_name if word in pname]) != 0: # instru. name
                    main_cand.append(s)
                    value=1
                ismaster.update({i:value})
        #             cand.remove(s)
            if len([word for word in sub_name if word in pname]) != 0:
                sub_cand.append(s)
                cand.remove(s)
            for cin in [c for c in list(s.select('Instrument')[0].children) if c != '\n']: # more than one instru. in this staff
                if cin.get('pitch') is not None:
                    if s in cand:
                        sub_cand.append(s)
                        cand.remove(s)
                    break
        mesg = {}
        pan = panDect(cand)
        ids = pan.keys()
        vels = velDect(addr,ids)
        pits = pitDect(addr,ids)
        rates = rateDect(addr,ids)
        area = areaDect(addr,ids)
        # print pan.keys(), area.keys(),ismaster1.keys(),ismaster1
        # print len(pan),len(vels),len(pits),len(rates),len(area),len(ismaster1)
        [mesg.update({a:[pan[a],vels[a],pits[a],rates[a],area[a],ismaster[a]]}) for a in pan.keys()]
    #   f = detMain(mesg)
        # print f
        #以下為判斷LONGNAME裡有main_name的歌寫到CSV
        if len([dd for dd in sorted(pan.keys()) if ismaster[dd]==1]) != 0:        
            for a in sorted([int(x) for x in mesg.keys()]):
                a=str(a)
                for b in range(len(mesg[a])):
                    fi.write(str(mesg[a][b])+',')
                fi.write('\n')
            fi.write('\n')
            print 'write:'+filename.decode('big5')
    except:
        print filename+',error'
fi.close()

write:16.mscx
write:2 Become 1.mscx
write:3 AM.mscx
write:409 In Your Coffeemaker.mscx
write:500 Miles.mscx
write:7 Things.mscx
write:A Lovers Concerto.mscx
write:A Song For You.mscx
write:A Warm Place.mscx
write:Adia.mscx
write:Afternoon Delight.mscx
write:Again.mscx
write:All I Ever Need Is You.mscx
write:All I Have to Do Is Dream.mscx
write:All Night Long.mscx
write:All That She Wants.mscx
write:Always Be My Baby.mscx
write:Always on my mind.mscx
write:Always somewhere.mscx
write:Always.mscx
write:America.mscx
write:American Idiot.mscx
write:American Pie.mscx
write:Amish Paradise.mscx
write:And I love you so.mscx
write:And Then He Kissed Me.mscx
write:And When I Die.mscx
write:Angels.mscx
write:Annies Song.mscx
write:Any Time You Need A Friend.mscx
write:Are you lonesome tonight.mscx
write:Armitage Shanks.mscx
write:Arthurs Theme.mscx
write:As Long As You Love Me.mscx
write:At last.mscx
write:At Your Side.mscx
Ave Maria.mscx,error
write:Baby Boy feat Sean Paul.mscx
Baby feat Ludacri